In [13]:
import cv2
import numpy
import os

def get_sigma(i):
    return 2 ** i

def decompose_to_energy_bands(image, b):
    energy_bands = {}
    energy_bands[0] = image
    for i in range(1, b + 1):
        energy_bands[i] = energy_bands[i - 1] - cv2.GaussianBlur(energy_bands[i - 1], (0, 0), get_sigma(i))
    return energy_bands


def m_func(energy_band, omega):
    interest_sum = 0
    n = len(omega)
    for i in range(n):
        pos = omega[i]
        interest_sum += energy_band[pos[0]][pos[1]]
    return 1.0 / (n - 1) * interest_sum


def compute_energy_value(i, omega, energy_bands):
    energy_band = energy_bands.get(i)
    m = m_func(energy_band, omega)
    n = len(omega)
    sum_of_square = 0
    for j in range(n):
        pos = omega[j]
        x = pos[0]
        y = pos[1]
        sum_of_square += (energy_band[x][y] - m) ** 2
    return (1.0 / (n-1) * sum_of_square) ** 0.5


def compute_ref_energy_value(i, omega, ref_images_energy_bands):
    sum_of_energy_value = 0
    r = len(ref_images_energy_bands)
    for j in range(r):
        sum_of_energy_value += compute_energy_value(i, omega, ref_images_energy_bands[i])
    return 1.0 / r * sum_of_energy_value



def compute_normalized_image(energy_bands, lambda_indicator, b):
    norm = []
    for i in range(1, b + 1):
        norm_i = numpy.array(energy_bands[i]) * lambda_indicator[i]
        if norm == []:
            norm = norm_i
        else:
            norm = numpy.add(norm, norm_i)
    return norm


# the entrance of the algorithm, image is a 2d array of pixel intensity
# ref_images is a list of 2d array (of pixel intensity), b is the band
# count, omega is a list of [x, y] where x,y stand for the pixel position
# of the interests region
def begin_enorm(image, ref_images, omega, b):
    # decompose all ref images
    ref_images_energy_bands = {}
    r = len(ref_images)
    for i in range(r):
        ref_images_energy_bands[i] = decompose_to_energy_bands(ref_images[i], b)

    # compute ref energy value
    ref_energy_values = {}
    for i in range(1, b + 1):
        ref_energy_values[i] = compute_ref_energy_value(i, omega, ref_images_energy_bands)

    lambda_indicator = {}

    i_norm = image
    counter = 0
    while counter < 30:
        cur_image = i_norm
        energy_bands = decompose_to_energy_bands(cur_image, b)
        for i in range(1, b + 1):
            energy_value = compute_energy_value(i, omega, energy_bands)
            lambda_indicator[i] = ref_energy_values[i] / energy_value
        i_norm = compute_normalized_image(energy_bands, lambda_indicator, b)
        counter += 1
        print(lambda_indicator)
        print(i_norm)
        print(counter)
        print(b)

    cv2.imwrite("norm_result.jpg", i_norm)

ref_images = []
for file in os.listdir("ref"):
    img = cv2.imread(os.path.join("ref", file)).astype(numpy.float32)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img = cv2.resize(img, (280, 280))[28:252, 28:252]
    cv2.imwrite(os.path.join("small_ref", file), img)
    ref_images.append(img)

orig_image = cv2.imread("test/covid/01E392EE-69F9-4E33-BFCE-E5C968654078.jpeg").astype(numpy.float32)
orig_image = cv2.cvtColor(orig_image, cv2.COLOR_BGR2GRAY)
omega = []
for i in range(224):
    for j in range(224):
        omega.append([i, j])

b = 5
begin_enorm(orig_image, ref_images, omega, b)


/Users/chihangwang/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:51: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.


{1: 2.7232402922490726, 2: 0.996396839576821, 3: 1.0729501722090204, 4: 1.0588184668603495, 5: 1.0984070462651674}
[[-1.03740833e+03  5.55970093e+02  4.94714081e+02 ...  1.29320145e+02
   1.41361359e+02  7.82195206e+01]
 [-1.00476526e+03  5.54166809e+02  4.23634552e+02 ...  4.81689568e+01
   3.97807579e+01 -2.32786798e+00]
 [-9.32790283e+02  6.06471802e+02 -9.12371948e+02 ...  3.76400414e+01
  -1.13175316e+01 -3.20902176e+01]
 ...
 [ 1.89761710e+00  1.78006327e+00  1.48827958e+00 ... -9.58649993e-01
   2.07874918e+00 -3.77262449e+00]
 [ 1.01865172e+00  9.36486363e-01  7.39371538e-01 ...  1.30283189e+00
   4.24971867e+00 -1.63187635e+00]
 [ 7.04605699e-01  6.37711287e-01  4.81303424e-01 ... -3.25572670e-01
   2.58913994e+00  3.64608622e+00]]
1
5
{1: 0.42328653450967174, 2: 0.15218563854121245, 3: 0.16382055111287688, 4: 0.16166274872819456, 5: 0.16770721747247017}
[[-1.13201868e+03  5.60784546e+02  4.73454010e+02 ...  1.10169113e+02
   1.21774139e+02  5.39268341e+01]
 [-1.09153601e+03  